# Spin chain with PBC

Using TEBD algorithm to simulate the time evolution of a system.

In [3]:
import quimb as qu
import quimb.tensor as qtn
import numpy as np
import sys

In [4]:
### PROBLEM PARAMETERS
L = 10       # chain length
omega = 1    # Rabi frequency
V = 1        # bath spins interaction strength
V_ = 1       # subsystem interaction strength
beta = 0.1   # inverse temperature for the bath

dims = [2]*L # overall space of L qbits

### Thermal bath
$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}
\newcommand{\Tr}{\text{Tr}}
$$

I need to create the bath B at a given teperature $\beta$ 
$$\rho=\frac{e^{-\beta H}}{Z}$$
exploiting an ancillary system Q.

I start by creating the purification of the $\beta=0$ system
$$\rho_0 = \bigg( \frac{1}{d} \hat{I} \bigg)^{\otimes L}$$
where
$$\frac{1}{d}\hat{I} = \Tr_Q \bigg[ \bigg( \sum_{\sigma} \frac{1}{\sqrt{d}}\ket{\sigma}_{B}\ket{\sigma}_{Q} \bigg) \bigg( \sum_{\sigma} \frac{1}{\sqrt{d}}\bra{\sigma}_{B}\bra{\sigma}_{Q} \bigg) \bigg]$$.

In [1]:
### CREATING THE PURIFICATION OF $\beta=0$ STATE

In [5]:
### BUILDING THE HAMILTONIAN(s)

# fisrt I build the operators
pauli_x = [qu.ikron(qu.pauli('X'), dims, i) for i in range(L)]

n_op = (1 + qu.pauli('Z'))/2
int_nn = [qu.ikron(n_op, dims, inds=[i,i+1]) for i in range(L-1)]
int_nn = qu.ikron(n_op, dims, inds=[0,L-1]) + int_nn # the first element is the 1-L interaction

# let me first build the bath hamiltonian
H_B = (omega/2)*sum(pauli_x[2:])
H_B += V*sum(int_nn[3:])

# and then add the rest
H = H_B + (omega/2)*sum(pauli_x[:2])
H += V_*sum(int_nn[:3])

In [53]:
### BUILDING THE SPIN CHAIN

# generating a random density matrix
rho_S = qu.gen.rand.rand_rho(4)

# thermal state for the bath
rho_B = qu.gen.states.thermal_state(H_B, beta)


In [6]:
int_nn

[[[2.  +0.j 0.5 +0.j 0.  +0.j ... 0.  +0.j 0.  +0.j 0.  +0.j]
  [0.5 +0.j 1.  +0.j 0.  +0.j ... 0.  +0.j 0.  +0.j 0.  +0.j]
  [0.  +0.j 0.  +0.j 2.  +0.j ... 0.  +0.j 0.  +0.j 0.  +0.j]
  ...
  [0.  +0.j 0.  +0.j 0.  +0.j ... 0.  +0.j 0.  +0.j 0.  +0.j]
  [0.  +0.j 0.  +0.j 0.  +0.j ... 0.  +0.j 0.  +0.j 0.  +0.j]
  [0.  +0.j 0.  +0.j 0.  +0.j ... 0.  +0.j 0.  +0.j 0.  +0.j]]

 [[2.  +0.j 0.5 +0.j 0.  +0.j ... 0.  +0.j 0.  +0.j 0.  +0.j]
  [0.5 +0.j 1.  +0.j 0.  +0.j ... 0.  +0.j 0.  +0.j 0.  +0.j]
  [0.  +0.j 0.  +0.j 2.  +0.j ... 0.  +0.j 0.  +0.j 0.  +0.j]
  ...
  [0.  +0.j 0.  +0.j 0.  +0.j ... 0.  +0.j 0.  +0.j 0.  +0.j]
  [0.  +0.j 0.  +0.j 0.  +0.j ... 0.  +0.j 0.  +0.j 0.  +0.j]
  [0.  +0.j 0.  +0.j 0.  +0.j ... 0.  +0.j 0.  +0.j 0.  +0.j]]

 [[2.  +0.j 0.5 +0.j 0.  +0.j ... 0.  +0.j 0.  +0.j 0.  +0.j]
  [0.5 +0.j 1.  +0.j 0.  +0.j ... 0.  +0.j 0.  +0.j 0.  +0.j]
  [0.  +0.j 0.  +0.j 2.  +0.j ... 0.  +0.j 0.  +0.j 0.  +0.j]
  ...
  [0.  +0.j 0.  +0.j 0.  +0.j ... 0.  +0.j 0.  +

In [1]:
from quimb import *

Simple conversion of data into Quimb objects.

In [2]:
data = [1, 2j, -3]

### Ket

In [3]:
qu(data, qtype='ket')

[[ 1.+0.j]
 [ 0.+2.j]
 [-3.+0.j]]

### Bra

In [4]:
qu(data, qtype='bra') # it conjugates the data!

[[ 1.-0.j  0.-2.j -3.-0.j]]

### Density operator

In [5]:
qu(data, qtype='dop')
#qu(data, qtype='dop', sparse=True)

[[ 1.+0.j  0.-2.j -3.-0.j]
 [ 0.+2.j  4.+0.j  0.-6.j]
 [-3.+0.j  0.+6.j  9.+0.j]]

Some basic operations in Quimb

In [6]:
psi = up()
psi.H       # hermitian conjugate
psi.H @ psi #inner product

[[1.+0.j]]

In [7]:
#better to use
expec(psi, psi)

1.0

In [8]:
X = pauli('X')
expec(psi, X) # operator expectation value

0j

## Tensoring

The tensor product can be done using kron() or the & overloead.

To sandwich operators with many Id one uses ikron()

For mor advanced stuff use pkron()

In [12]:
dims = [2]*10
X = pauli('X')
IIIXXIIIII = ikron(X, dims, inds=[3, 4])
IIIXXIIIII.shape

(1024, 1024)

In [10]:
dims

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]